In [1]:
!pip install datasets pandas matplotlib seaborn nltk

import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.tokenize import word_tokenize
from datasets import load_dataset

nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
!git clone https://github.com/Yale-LILY/QMSum.git

Cloning into 'QMSum'...
remote: Enumerating objects: 809, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 809 (delta 1), reused 0 (delta 0), pack-reused 804 (from 1)
Receiving objects: 100% (809/809), 13.76 MiB | 16.35 MiB/s, done.
Resolving deltas: 100% (446/446), done.


In [6]:
!ls /content/QMSum/data/ALL

jsonl  test  train  val


In [10]:
# !ls /content/QMSum/data/ALL/jsonl/train

!find /content/QMSum/data/ALL -name "*.jsonl"

/content/QMSum/data/ALL/jsonl/test.jsonl
/content/QMSum/data/ALL/jsonl/val.jsonl
/content/QMSum/data/ALL/jsonl/train.jsonl


In [14]:
qmsum_train_df = load_qmsum('/content/QMSum/data/ALL/jsonl/train.jsonl')
qmsum_val_df   = load_qmsum('/content/QMSum/data/ALL/jsonl/val.jsonl')
qmsum_test_df  = load_qmsum('/content/QMSum/data/ALL/jsonl/test.jsonl')

qmsum_train_df['split'] = 'train'
qmsum_val_df['split'] = 'val'
qmsum_test_df['split'] = 'test'

qmsum_df = pd.concat([qmsum_train_df, qmsum_val_df, qmsum_test_df], ignore_index=True)
qmsum_df.head()


,meeting_id,query,summary,spans,split
0,None,Summarize the whole meeting.,"Firstly, Project Manager reviewed the overall ...",[],train
1,None,How Did Project Manager and User Interface int...,Project Manager introduced that the prototype ...,"[[0, 39]]",train
2,None,How did Marketing design the product evaluation?,Marketing had some evaluation criteria in mind...,"[[54, 61]]",train
3,None,What did the team discuss during the product e...,"Generally speaking, the team agreed that the p...","[[64, 189]]",train
4,None,How did the team evaluate the product about it...,"To start with, the team agreed that the remote...","[[190, 249]]",train


In [15]:
print("Total rows:", len(qmsum_df))
print("Unique meetings:", qmsum_df['meeting_id'].nunique())
print("Query types:", qmsum_df.columns)  # make sure 'query', 'summary', 'spans' exist

Total rows: 1810
Unique meetings: 0
Query types: Index(['meeting_id', 'query', 'summary', 'spans', 'split'], dtype='object')


In [16]:
from nltk.tokenize import word_tokenize


qmsum_df["query_tokens"] = qmsum_df["query"].apply(lambda x: len(word_tokenize(x)))
qmsum_df["summary_tokens"] = qmsum_df["summary"].apply(lambda x: len(word_tokenize(x)))


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [4]:
# Load from local GitHub clone? or via huggingface loader?
# Option A: Local JSONL files from Yale-LILY/QMSum
def load_qmsum(path):
    items = []
    for line in open(path):
        obj = json.loads(line)
        for q in obj['general_query_list'] + obj['specific_query_list']:
            items.append({
                'meeting_id': obj.get('uid', obj.get('meeting_id', None)),
                'query': q['query'],
                'summary': q['answer'],
                'spans': q.get('relevant_text_span', [])
            })
    return pd.DataFrame(items)

# qmsum_df = load_qmsum('path/to/QMSum/data/ALL/train.jsonl')


In [5]:
qmsum_df = load_qmsum('/content/QMSum/data/ALL/train.jsonl')

FileNotFoundError: [Errno 2] No such file or directory: '/content/QMSum/data/ALL/train.jsonl'